In [34]:
import pandas as pd
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
import rasterio
from rasterio.crs import CRS
from rasterio.transform import from_bounds
from shapely.geometry import box, Point, Polygon, MultiPolygon
from scipy.spatial import cKDTree
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,WhiteKernel,ConstantKernel
from flood_density.plots import convert_kml_to_geojson
from flood_density.preprocess import extract_city_bounds, extract_bounds_polygon, polygon_to_geodataframe, convert_gdf_to_geojson, extract_city_data, coordinates_to_box,points_geocoordinates,convert_points_in_gdf, extract_city_bounds_from_dataframe_to_geodataframe

from typing import List, Tuple

CRS_4326 = 4326


# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [ ]:
convert_kml_to_geojson("laplata_cascourbano.kml", "salida.geojson")

# Etapa 2: Obtener las coordenadas de los puntos de la ciudad


In [ ]:
#¿Cómo obtengo bounds_dict? son coordenadas de la ciudad de La Plata Desde el archivo geojson

In [ ]:
# Cargar el archivo GeoJSON del plot
gdf_la_plata = gpd.read_file("salida.geojson")

In [ ]:
#Extraemos las coordenadas geográficas de la ciudad 
city_bounds_coordinates= extract_city_bounds(gdf_la_plata)
city_bounds_coordinates

In [ ]:
polygon_coords = extract_bounds_polygon(city_bounds_coordinates)

In [ ]:
geo_polygon = polygon_to_geodataframe(polygon_coords, CRS_4326)


In [ ]:
convert_gdf_to_geojson(geo_polygon, "polygon.geojson")

# Etapa 3 : Obtener datos de densidad poblacional de la ciudad de La Plata.

Primero extraemos las filas que pertenecen a la ciudad de La Plata(sigue siendo un .csv), luego convertimos ese dataframe en un geodataframe.

In [ ]:
city_bounds_coordinates

In [ ]:
coordinates_to_box(city_bounds_coordinates)

In [ ]:
# Create GeoDataFrame from the bounding box
#gdf_la_plata = gpd.GeoDataFrame({'geometry': [bbox]}, crs='EPSG:4326')

# Display the result
#gdf_la_plata.to_file("la_plata_bounds.geojson", driver='GeoJSON')

# Etapa 4 : Transformar datos de densidad poblacional de la ciudad de La Plata.

In [ ]:
#Levantamos el dataset completo
df_arg = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")

In [ ]:
# Extraemos las filas referidas a la ciudad de La Plata
df_lp_coordinates = extract_city_data(df_arg,'la_plata',city_bounds_coordinates)

In [ ]:
#Leemos los datos específicos de la ciudad de La Plata
df_lp = pd.read_csv('la_plata_population_2020.csv')


In [ ]:
# Creamos un polígono con los puntos referidos a la ciudad
points_for_density = points_geocoordinates(df_lp)

In [ ]:
points_for_density

In [ ]:
# Guardamos los puntos geográficos en una columna (dataframe)
df_geometry = convert_points_in_gdf(points_for_density,CRS_4326)

In [ ]:
df_geometry

In [ ]:
# Agregar columna geometry
df_lp_coordinates.loc[:, 'geometry'] = df_geometry['geometry'].values


In [ ]:
#Convertimos el dataframe en un geodataframe
#extract_city_bounds_from_dataframe_to_geodataframe(df_lp_coordinates)

In [ ]:
#Convertimos df_lp_coordinates a un geojson
#export_to_geojson(gdf)

In [ ]:
def density_city_plot(gdf: gpd.GeoDataFrame, )

# Visualización de la densidad poblacional de La Plata
fig, axes = plt.subplots(figsize=(10, 10))

# Gráfico de dispersión geográfica
scatter = axes.scatter(
    gdf_densidad['X'], gdf_densidad['Y'], c=gdf_densidad['Z'],
    cmap='inferno', alpha=0.6, s=10
)
axes.set_xlabel('Longitud')
axes.set_ylabel('Latitud')
axes.set_title('Distribución Geográfica de Densidad Poblacional de La Plata')

# Ajustar límites del gráfico según los límites de La Plata
axes.set_xlim(bounds['x_min'], bounds['x_max'])
axes.set_ylim(bounds['y_min'], bounds['y_max'])


plt.colorbar(scatter, ax=axes, label='Densidad')
plt.tight_layout()
plt.savefig('densidad_la_plata.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
def clip_density_to_urban_area(gdf_density: gpd.GeoDataFrame, gdf_geographical: gpd.GeoDataFrame) -> gpd.GeoDataFrame: 
    # Usar uniones espaciales para mantener solo puntos dentro del casco urbano
    points_in_casco = gpd.sjoin(gdf_density, gdf_geographical, how='inner', predicate='intersects')
    
    # Limpiar columnas duplicadas del join 
    points_in_casco = points_in_casco.drop(columns=[col for col in points_in_casco.columns if col.endswith('_right')]) 
    
    return points_in_casco

In [ ]:
casco_urbano = clip_density_to_urban_area(gdf_densidad, gdf_geographical= gdf)

In [ ]:
casco_urbano

In [ ]:
# Convertir a Web Mercator (sist. de proyección cartográfica EPSG 3857)
casco_utm = casco_urbano.to_crs(epsg=3857)

In [ ]:
# Crear un GeoDataFrame con los límites del área de La Plata en EPSG:4326
gdf_bounds_4326 = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

# Transformar al CRS del plot
gdf_bounds_utm = gdf_bounds_4326.to_crs(casco_utm.crs)

# Obtener límites en coordenadas proyectadas
bounds_proj = gdf_bounds_utm.total_bounds  # [xmin, ymin, xmax, ymax]

In [ ]:
# Plotear usando un mapa base más clara y con mayor contraste
fig, ax = plt.subplots(figsize=(10, 10))

casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',  
               alpha=0.8,    
               edgecolor='black',
               linewidth=2,
               ax=ax,
               legend=True,
               legend_kwds={'shrink': 0.7})

# Mapa base
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(), 
                source=ctx.providers.CartoDB.Positron,  # Fondo más claro
                alpha=0.7)  

ax.set_title('Densidad Poblacional - Casco Urbano La Plata', fontsize=16)
ax.axis('on')

# Ajustar límites del gráfico según los límites de La Plata
ax.set_xlim(bounds_proj[0], bounds_proj[2])
ax.set_ylim(bounds_proj[1], bounds_proj[3])

plt.tight_layout()
plt.show()

In [ ]:
# Visualización de la densidad poblacional de La Plata usando kriging

# Extraer coordenadas de los centroides,y valores desde el GeoDataFrame
coords =np.array([[point.x, point.y] for point in casco_utm.geometry.centroid]) 
values_density = casco_urbano['densidad_poblacional_lp'].values

print(f"Puntos de datos: {len(coords)}")    
print(f"Rango de valores: {values_density.min():.2f} - {values_density.max():.2f}")

# Crear grid de puntos para interpolación
grid_x, grid_y = np.mgrid[bounds_proj[0]:bounds_proj[2]:150j, bounds_proj[1]:bounds_proj[3]:150j]
grid_coords = np.column_stack([grid_x.ravel(), grid_y.ravel()])

#Definir kernel para el modelo de kriging
kernel = (ConstantKernel(1.0) * RBF(length_scale=1000.0) + 
          WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e3)))

# Crear modelo de kriging
gpr = GaussianProcessRegressor(kernel=kernel,
                                n_restarts_optimizer=10,
                                alpha=1e-04,
                                normalize_y=True)

# Entrenar el modelo
gpr.fit(coords, values_density)

#Predecir en el grid
grid_2_kriging, pred_std = gpr.predict(grid_coords, return_std=True)
grid_z = grid_2_kriging.reshape(grid_x.shape)
grid_uncertainty = pred_std.reshape(grid_x.shape)

# Visualización de la superficie Kriging interpolada
fig, ax = plt.subplots(figsize=(10, 10))

# Plotear superficie Kriging interpolada
contour = ax.contourf(grid_x, grid_y, grid_z, levels=30, cmap='viridis', alpha=0.5)

# Plotear los polígonos originales con bordes
casco_utm.plot(column='densidad_poblacional_lp', 
               cmap='viridis',
               alpha=0.5,
               edgecolor='black',
               linewidth=1.0,
               ax=ax)

# Plotear los puntos centroides
scatter = ax.scatter(coords[:, 0], coords[:, 1],
                    c=values_density,
                    cmap='viridis',
                    s=30,
                    edgecolors='black',
                    linewidths=1,
                    zorder=5)

# Ajustar límites del gráfico según los límites de La Plata
ax.set_xlim(bounds_proj[0], bounds_proj[2])
ax.set_ylim(bounds_proj[1], bounds_proj[3])

#Mapa base más sutil
ctx.add_basemap(ax, crs=casco_utm.crs.to_string(),
               source=ctx.providers.CartoDB.Positron,
               alpha = 0.9)

# Agregar colorbar para el scatter
plt.colorbar(scatter, ax=ax, label='Densidad poblacional',shrink=0.7, aspect=25)

# Agregar título y etiquetas
ax.set_title('Interpolación Kriging - Densidad Poblacional La Plata')
ax.set_xlabel('X (UTM)')
ax.set_ylabel('Y (UTM)')

plt.tight_layout()
plt.show()

In [ ]:
def export_to_geotiff(grid_data: np.ndarray, gdf: gpd.GeoDataFrame, filename: str, crs_epsg: int = 3857):

     # Crear un GeoDataFrame con los límites del área de La Plata en EPSG:4326
    gdf_bounds_4326 = gpd.GeoDataFrame(geometry=[bbox], crs="EPSG:4326")

    # Transformar al CRS del plot
    gdf_bounds_utm = gdf_bounds_4326.to_crs(casco_utm.crs)

    # Obtener límites en coordenadas proyectadas
    bounds_proj = gdf_bounds_utm.total_bounds  # [xmin, ymin, xmax, ymax]
    height, width = grid_data.shape

    # Calcula la transformación georreferenciada
    transform = from_bounds(bounds_proj[0], bounds_proj[1], bounds_proj[2], bounds_proj[3], width, height)

    with rasterio.open(
        filename,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=grid_data.dtype,
        crs=CRS.from_epsg(crs_epsg),
        transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(grid_data, 1)
        dst.set_band_description(1, 'Densidad Poblacional Kriging')



In [ ]:
# Exportar superficie interpolada
export_to_geotiff(grid_z, gdf_bounds_4326, 'kriging_densidad_poblacional.tif', 3857)

# Etapa 5 : superposición de mapas de densidad y peligrosidad de inundación de la ciudad de La Plata (QGIS).